# Summarize wifi results

In [4]:
import torch
import numpy as np

tot1

In [11]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId1_seed%d_idim17_DA_Poolnn_ML_warmup200_MLP_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden17_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('poolNN')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId1_seed%d_idim17_DA_Infer_JMMD_ML_warmup200_MLP_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden17_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMMD')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId1_seed%d_idim17_DA_Infer_JMMD_DAG_m1_ML_warmup200_gp10.00_sn0_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMM_DAG')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId1_seed%d_idim17_DA_Infer_JMMD_DAG_ML_warmup200_PDAG_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMMD_DAG')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

poolNN
0.44288565946598213 0.020894910050682978
0.38844756112446827 0.02086963521905495
JMMD
0.45764487079925675 0.022054343715207193
0.43218179542286306 0.019986072785455094
JMM_DAG
0.4555635799580838 0.024134913257665837
0.4186122034012321 0.030024850302709183
JMMD_DAG
0.4515743909593656 0.020642778242433348
0.4115586819007856 0.015064046021167728


tot2

In [15]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId2_seed%d_idim16_DA_Poolnn_ML_warmup200_MLP_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden16_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('poolNN')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId2_seed%d_idim16_DA_Infer_JMMD_ML_warmup200_MLP_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden16_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMMD')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId2_seed%d_idim16_DA_Infer_JMMD_DAG_m1_ML_warmup200_gp10.00_sn0_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMMD_DAG')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId2_seed%d_idim16_DA_Infer_JMMD_DAG_ML_warmup200_PDAG_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMMD_DAG')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

poolNN
0.46087077428005035 0.034802856627661055
0.36260250441396946 0.027985622617256626
JMMD
0.47097529301375596 0.050520309737478494
0.4264494769424931 0.05700493226554852
JMM_DAG
0.46859216900468265 0.03441041470728024
0.3580530761447246 0.04162150993734839
JMMD_DAG
0.463213579280268 0.032877042760163844
0.3729577771972435 0.039862812343788516


tot3

In [17]:
num_rep = 10
acc_best = np.zeros(num_rep)
acc_final = np.zeros(num_rep)

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId3_seed%d_idim16_DA_Poolnn_ML_warmup200_MLP_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden16_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('poolNN')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId3_seed%d_idim16_DA_Infer_JMMD_ML_warmup200_MLP_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden16_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMMD')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId3_seed%d_idim16_DA_Infer_JMMD_DAG_m1_ML_warmup200_gp10.00_sn0_DAG_Generator_MLP_Classifier_Diter1_AC_weight1.00_SRC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMMD_DAG')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

for i in range(num_rep):
    file_name = '../weights/DatasetWifi/_tarId3_seed%d_idim16_DA_Infer_JMMD_DAG_ML_warmup200_PDAG_Generator_MLP_Classifier_AC_weight1.00_TAR_weight0.20_useMB1_mlp_nodes20_bs210_Glr2.0e-03_Dlr2.0e-03_numDomain3_dimDomain1_dimHidden1_state.pkl' % (i+1)
    x = torch.load(file_name)
    acc_best[i] = x['best_score']
    acc_final[i] = x['final_score']
print('JMMD_DAG')
print(acc_best.mean(), acc_best.std())
print(acc_final.mean(), acc_final.std())

poolNN
0.4044003320266933 0.021106772229728354
0.37005292345326596 0.02121543081464533
JMMD
0.43818185852417174 0.03216136572554337
0.4072916733522164 0.029202692128646568
JMMD_DAG
0.3861774494759768 0.0314601210944744
0.34022227528695614 0.03634783138695622
JMMD_DAG
0.4042683062328097 0.023116424966342992
0.3647919706716461 0.03706208766988439
